In [11]:

import os
import glob
import sys
sys.path.append('../../')
from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm
from vollseg import Augmentation3D
from vollseg.helpers import crappify_flou_G_P
from pathlib import Path

In [12]:
ImageDir =  '/Users/aimachine/Mari/Raw/'
LabelDir = '/Users/aimachine/Mari/RealMask/'

In [21]:
Raw_path = os.path.join(ImageDir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort


RealRaw_path = os.path.join(LabelDir, '*tif')
RealfilesRaw = glob.glob(RealRaw_path)
RealfilesRaw.sort
mu = 0
sigma = 5
for fname in filesRaw:


           for Realfname in RealfilesRaw:

                  name = os.path.basename(os.path.splitext(fname)[0])
                  realname = os.path.basename(os.path.splitext(Realfname)[0])

                  if ( name == realname): 

                            #Read image        
                            image = imread(fname)
                            y = imread(Realfname)
                            #Get Name
                            x = image
                            crappify_flou_G_P(x, y, mu, sigma, ImageDir, LabelDir, name)

In [22]:
Raw_path = os.path.join(ImageDir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

Label_path = os.path.join(LabelDir, '*tif')
filesLabel = glob.glob(Label_path)
filesLabel.sort

Data = []
Label = []
for fname in filesRaw:

    for secondfname in filesLabel:

        Name = os.path.basename(os.path.splitext(fname)[0])
        LabelName = os.path.basename(os.path.splitext(secondfname)[0])
        if Name == LabelName:
            image = imread(fname)
            Data.append(image)
            labelimage = imread(secondfname)
            Label.append(labelimage)   
            
Data = np.asarray(Data)
Label = np.asarray(Label)
   
print(Data.shape)


(160, 20, 200, 200)


In [15]:

AugmentData = Augmentation3D(rotate_axis=2, rotate_angle = 30)
AugImageLabel = AugmentData.build(data=Data, label=Label, batch_size = Data.shape[0])



In [16]:
#Save the shifts
AugPair = np.asarray(next(AugImageLabel))
Count = 0
for i in range(0, AugPair.shape[1]):
     Name = 'rotateax2' + str(Count)
     imwrite(ImageDir + '/' + Name + '.tif', AugPair[0,i,:,:].astype('float32'))
     imwrite(LabelDir + '/' + Name + '.tif', AugPair[1,i,:,:].astype('uint16'))
     Count = Count + 1

In [19]:

AugmentData = Augmentation3D(rotate_axis=1, rotate_angle = 'random')
AugImageLabel = AugmentData.build(data=Data, label=Label, batch_size = Data.shape[0])



In [20]:
#Save the flips
AugPair = np.asarray(next(AugImageLabel))
Count = 0
for i in range(0, AugPair.shape[1]):
     Name = 'rotateax1' + str(Count)
     imwrite(ImageDir + '/' + Name + '.tif', AugPair[0,i,:,:].astype('float32'))
     imwrite(LabelDir + '/' + Name + '.tif', AugPair[1,i,:,:].astype('uint16'))
     Count = Count + 1